In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import ascii
from collections import Counter
import time
rcParams['font.family'] = 'serif'
% matplotlib

Using matplotlib backend: TkAgg


In [2]:
converters = {'col1': [ascii.convert_numpy(np.uint64)],
              'col2': [ascii.convert_numpy(np.uint64)],
              'col3': [ascii.convert_numpy(np.float64)],
              'col4': [ascii.convert_numpy(np.float64)],
              'col5': [ascii.convert_numpy(np.uint)]}


from astropy.io import ascii
data = ascii.read('DL07_milimill_HOD.csv', converters=converters)

In [3]:
lenght = len(data)
IDs, fofid, stellarmass, centralmvir, Type = np.zeros(lenght), np.zeros(lenght), np.zeros(lenght), np.zeros(lenght), np.zeros(lenght)

for i, line in enumerate(data):
    IDs[i], fofid[i], stellarmass[i], centralmvir[i], Type[i] = line  
    fofid[i] %= 1e9

fofid = fofid.astype(np.int64)

In [ ]:
IDs_mass = {}
for i in range(len(fofid)):
    IDs_mass[str(fofid)]


print fofid[:10]
print centralmvir[:10]

In [4]:
NBIN = 30
halo_masses = np.log10(centralmvir) + 10
hmin, hmax = np.min(halo_masses), np.max(halo_masses) + 1e-10
mass_bins = ((halo_masses - hmin)/(hmax - hmin) * NBIN).astype(int)

bin_width = (hmax - hmin)/NBIN
bins = np.linspace(hmin, hmax, NBIN) + bin_width *.5

In [5]:
def counter (L):
    Nhalos = len(L)
    Ngal, Ncen, Nsat = 0, 0, 0
    for i in range(Nhalos):
        #L_array = np.array(L[i])
        #if len(L[i]) == 1:
            #Ngal += 1
        #    Ncen += 1
        #    continue
        Ngal += len(L[i]) # Solo galaxias con M > 10^9 Msun
        sat_mask = L[i] != 0
        Nsat += len((L[i])[sat_mask])
        Ncen += int(np.any(L[i] == 0))
        
    return Nhalos, Ngal, Ncen, Nsat




In [11]:
def counter (T, F, M):
    Halos = len(T)
    Ngal, Ncen, Nsat = 0, 0, 0
    for i in range(Halos):
        
        T_array = np.array(T[i])
        M_array = np.array(M[i])
        F_array = np.array(F[i])
        
        mass_mask = M_array > .1
        T_array = T_array[mass_mask]
        F_array = F_array[mass_mask]
        
        sat_mask = T_array != 0
        Nsat += len(T_array[sat_mask])
        Ncen += int(np.any(T_array == 0))
        Ngal += len(T_array)
    return Halos, Ngal, Ncen, Nsat, F_array

In [7]:
def ToList (fofid, Type, stellarmass):
    
    fofid_list = []
    type_list = []
    mass_list = []
    C = Counter(fofid)
    
    for FOFID in C.keys():
        
        mask_fofid = fofid == FOFID
        temp_fofid = fofid[mask_fofid]
        temp_Type = Type[mask_fofid]
        temp_stellarmass = stellarmass[mask_fofid]
        
        type_list.append(temp_Type)
        fofid_list.append(temp_fofid)  
        mass_list.append(temp_stellarmass)

    return fofid_list, type_list, mass_list


In [12]:
from itertools import groupby
from collections import Counter

Halos = np.zeros(NBIN)
Galaxies = np.zeros(NBIN)
Cent = np.zeros(NBIN)
Sats = np.zeros(NBIN)

for i in range(NBIN):
    
    # Índices correspondientes al bin de masa
    idx = np.where(mass_bins == i)[0]

    # Seleccionamos sólo los valores que corresponden al bin
    bin_halomasses = halo_masses[idx]
    bin_stellarmass = stellarmass[idx]
    bin_fofid = fofid[idx]
    bin_type = Type[idx]

    FOFID_list, TYPE_list, MASS_list = ToList(bin_fofid, bin_type, bin_stellarmass)
    #print len(FOFID_list), len(TYPE_list)
    
    Halos[i], Galaxies[i], Cent[i], Sats[i], FOFID = counter(TYPE_list, FOFID_list, MASS_list)
    
    #if i == 3:
    #    print FOFID_list
    #    print TYPE_list
        #print sort_fofid
        #print sort_type
        #print sort_stellarmass

    
    
    

In [ ]:
5/2

In [ ]:
FOFID_list

In [13]:
Galaxies += 1e-10
Cent += 1e-10
Sats += 1e-10
HOD = Galaxies/Halos
Cent /= Halos
Sats /= Halos

np.savetxt("HOD_halo_method.txt", np.array([bins, HOD]).T)

In [ ]:
bins2, HOD2, cent, sats = np.loadtxt("HOD.txt", unpack = True)


In [ ]:
plt.plot(bins, np.log10(HOD), 'k.', label = 'All')
plt.plot(bins, np.log10(Cent), 'r--', label = 'Centrals')
plt.plot(bins, np.log10(Sats), 'b--', label = 'Sats')
#plt.annotate(r"$M > 10^9 M_{\odot}$", xy=(80, 270), xycoords='figure pixels')
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$")
plt.ylabel(r"$log(N)$")
plt.ylim(-2,2)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left', frameon = False, prop={'size':10})
plt.show()

In [ ]:
# El problema es tratar de guardar las listas
# Simplemente lo podemos guardar en un archivo de texto...

In [ ]:
"""
start_time = time.time()
ROWS = len(IDs)
HOD = np.zeros(NBIN)
Centrals = np.zeros(NBIN)
Sats = np.zeros(NBIN)
Halos = np.zeros(NBIN)

mass_mask = (stellarmass > .1).astype(int)
labels = ((log_centralmvir-min_masslog)/(max_masslog-min_masslog)*NBIN).astype(int)
Sats_mask = Type.astype(bool) 

for i in range(ROWS):
    label = labels[i]
    sat = Sats_mask[i]
    Halos[label] += 
    HOD[label] += mass_mask[i]
    Sats[label] += int( sat == True)
    Centrals[label] += int( (sat == False)  )
    
print("--- %3fs: loaded ---" % (time.time() - start_time))    

mask = (labels < 0) | (labels >= NBIN)
IDs, fofid, Type = IDs[~mask], fofid[~mask], Type[~mask]
stellarmass, log_centralmvir = stellarmass[~mask], log_centralmvir[~mask]
labels = labels[~mask]

start_time = time.time()
ROWS = len(IDs)
HOD = np.zeros(NBIN)
Centrals = np.zeros(NBIN)
Sats = np.zeros(NBIN)

for i in range(ROWS):
    
    halo_mass = log_centralmvir[i]
    Sats_mask = Type[i].astype(bool) 
    mass_mask = stellarmass[i] > .1
    label = ((halo_mass-min_masslog)/(max_masslog-min_masslog)*NBIN).astype(int)
    
    HOD[label] += mass_mask.astype(int)
    Sats[label] += ((mass_mask) & (Sats_mask)).astype(int)
    Centrals[label] += ((mass_mask) & (not Sats_mask)).astype(int)    
    
print("--- %3fs: loaded ---" % (time.time() - start_time)) 




#converters = {0: np.uint64, 1: np.int64, 2: np.float64, 3: np.float64, 4: np.uint8}
#IDs, fofid, stellarmass, centralmvir, Type = np.loadtxt('DL07_milimill_HOD.csv',converters=converters, unpack = True)

"""